# Load a local ONNX model and do inference

## Load model

In [13]:
import onnx
import onnxruntime as ort
from transformers import DistilBertTokenizer
import json

# Load the ONNX model
model_path = "../onnxModels/model1/"
onnx_model = onnx.load(model_path+"model.onnx")

# Load configuration
with open(model_path+"config.json", "r") as f:
    config = json.load(f)

id2label = config.get("id2label", {})
print(f"id2label mapping: {id2label}")


# Check the model
onnx.checker.check_model(onnx_model)

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("../onnxModels/model1")

# Create an inference session
session = ort.InferenceSession(model_path+"model.onnx")

print("Model loaded and checked successfully.")

id2label mapping: {'0': 'non-adult', '1': 'adult'}
Model loaded and checked successfully.


In [15]:
import numpy as np

# Define a random input string
input_text = "Porn."

# Tokenize the input text
tokenized_input = tokenizer(
    input_text,
    return_tensors="np",  # Convert output to numpy arrays
    padding=True,
    truncation=True,
    max_length=512  # Ensure the sequence length matches the model's requirements
)

# Extract input IDs and attention mask from the tokenized input
input_ids = tokenized_input["input_ids"]
attention_mask = tokenized_input["attention_mask"]

# Prepare the inputs for the ONNX model
onnx_inputs = {
    "input_ids": input_ids.astype(np.int64),  # Convert to int64 as required by ONNX
    "attention_mask": attention_mask.astype(np.int64)
}

# Run inference
outputs = session.run(None, onnx_inputs)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

# The outputs variable contains logits. Apply softmax to convert logits to probabilities
logits = outputs[0]
# use softmax to convert logits to probabilities
probabilities = softmax(logits)
print(probabilities)

# Get the predicted label (index with the highest probability)
predicted_label_index = np.argmax(probabilities, axis=1)[0]
print(predicted_label_index)


# Get the label using the index
predicted_label = id2label[str(predicted_label_index)]  # Note: JSON keys are strings
print(f"Input Text: {input_text}")
print(f"Predicted Label: {predicted_label}")
print(f"Probabilities: {probabilities}")

[[1.1525081e-04 9.9988472e-01]]
1
Input Text: Porn.
Predicted Label: adult
Probabilities: [[1.1525081e-04 9.9988472e-01]]
